# Function

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
import re


def fetch_and_append_census_data(html_url):
    response = requests.get(html_url)

    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table')

    geo_title = soup.find('div', id='geo_title').text.strip()

    data_rows = table.find_all('tr', class_='value_row')

    data = []
    for row in data_rows:
        cols = row.find_all('td')
        t_col_1 = row.find('th', class_='t_col_1')
        if t_col_1:
            category = t_col_1.text.strip()

        table_id = table.get('id') if table else ''

        if len(cols) == 2:
            title = cols[0].text.strip()
            estimate_moe = cols[1].text.strip().split()
            estimate = estimate_moe[0]
            moe = estimate_moe[1] if len(estimate_moe) > 1 else ''
            data.append([table_id, category, title, estimate, moe])

    df_new = pd.DataFrame(data, columns=['Topic', 'Subject', 'Title', 'District 01 Estimate', 'District 01 MOE'])

    estimate_value = df_new.at[0, 'District 01 Estimate']
    moe_value = df_new.at[0, 'District 01 MOE']
    combined_value = f'{estimate_value}({moe_value.strip("()")})'

    df_new.at[0, 'District 01 Estimate'] = combined_value
    df_new.at[0, 'District 01 MOE'] = ''

    return df_new

In [2]:
def replace_placeholders_in_df(df, api_url):
    response = requests.get(api_url)
    api_response = json.loads(response.text)

    def replace_placeholders(text, api_response):
        placeholders = re.findall(r'\$\{([^}]+)\}', text)
        for placeholder in placeholders:
            value = api_response.get(placeholder, '')
            text = text.replace('${%s}' % placeholder, value)
        return text

    df = df.applymap(lambda x: replace_placeholders(x, api_response) if isinstance(x, str) else x)

    if 'District 01 Estimate' in df.columns:

      df[['District 01 Estimate', 'District 01 MOE']] = df['District 01 Estimate'].str.extract(r'([^()]+)\(([^)]+)\)', expand=True)
      df['District 01 MOE'] = df['District 01 MOE'].fillna('')
      df['District 01 MOE'] = df['District 01 MOE'].apply(lambda x: f"(+/-{x.strip()})" if x else '')

    return df

In [3]:
def saving_file(df, output_filename='scraped_data.xlsx'):
    if os.path.exists(output_filename):
        with pd.ExcelWriter(output_filename, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    else:
        df.to_excel(output_filename, index=False)
    print(f"Data written/appended to {output_filename}")

In [4]:
html_url_people = 'https://www.census.gov/mycd/application/templates/99_mcd_people.html'
html_url_employment = 'https://www.census.gov/mycd/application/templates/99_mcd_employment.html'
html_url_housing = 'https://www.census.gov/mycd/application/templates/99_mcd_housing.html'
html_url_income = 'https://www.census.gov/mycd/application/templates/99_mcd_financial.html'
html_url_education = 'https://www.census.gov/mycd/application/templates/99_mcd_education.html'
html_url_business = 'https://www.census.gov/mycd/application/templates/99_mcd_business.php?stateId=05&districtId=02'

api_url_people = 'https://www.census.gov/mycd/application/bin/functs_easystats.php?call=get_values&geo_type=CONGRESSIONAL_DISTRICT&geo_level_1=05&geo_level_2=01&url=https%3A%2F%2Fapi.census.gov%2Fdata%2F2023%2Facs%2Facs1%2Fprofile&tableid=99_mcd_people&key=f4a93d15173229253a4f234727b2902053f61bbd%3Bmycd'
api_url_employment = 'https://www.census.gov/mycd/application/bin/functs_easystats.php?call=get_values&geo_type=CONGRESSIONAL_DISTRICT&geo_level_1=05&geo_level_2=02&url=https%3A%2F%2Fapi.census.gov%2Fdata%2F2023%2Facs%2Facs1%2Fprofile&tableid=99_mcd_employment&key=f4a93d15173229253a4f234727b2902053f61bbd%3Bmycd'
api_url_housing = 'https://www.census.gov/mycd/application/bin/functs_easystats.php?call=get_values&geo_type=CONGRESSIONAL_DISTRICT&geo_level_1=05&geo_level_2=02&url=https%3A%2F%2Fapi.census.gov%2Fdata%2F2023%2Facs%2Facs1%2Fprofile&tableid=99_mcd_housing&key=f4a93d15173229253a4f234727b2902053f61bbd%3Bmycd'
api_url_income = 'https://www.census.gov/mycd/application/bin/functs_easystats.php?call=get_values&geo_type=CONGRESSIONAL_DISTRICT&geo_level_1=05&geo_level_2=02&url=https%3A%2F%2Fapi.census.gov%2Fdata%2F2023%2Facs%2Facs1%2Fprofile&tableid=99_mcd_financial&key=f4a93d15173229253a4f234727b2902053f61bbd%3Bmycd'
api_url_education = 'https://www.census.gov/mycd/application/bin/functs_easystats.php?call=get_values&geo_type=CONGRESSIONAL_DISTRICT&geo_level_1=05&geo_level_2=02&url=https%3A%2F%2Fapi.census.gov%2Fdata%2F2023%2Facs%2Facs1%2Fprofile&tableid=99_mcd_education&key=f4a93d15173229253a4f234727b2902053f61bbd%3Bmycd'


html_df_people = fetch_and_append_census_data(html_url_people)
html_df_employment = fetch_and_append_census_data(html_url_employment)
html_df_housing = fetch_and_append_census_data(html_url_housing)
html_df_income = fetch_and_append_census_data(html_url_income)
html_df_education = fetch_and_append_census_data(html_url_education)
# html_df_business = fetch_and_append_census_data(html_url_business)


In [5]:
final_df_people = replace_placeholders_in_df(html_df_people, api_url_people)
final_df_employment = replace_placeholders_in_df(html_df_employment, api_url_employment)
final_df_housing = replace_placeholders_in_df(html_df_housing, api_url_housing)
final_df_income = replace_placeholders_in_df(html_df_income, api_url_income)
final_df_education = replace_placeholders_in_df(html_df_education, api_url_education)

<ipython-input-2-50d803100278>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: replace_placeholders(x, api_response) if isinstance(x, str) else x)
<ipython-input-2-50d803100278>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: replace_placeholders(x, api_response) if isinstance(x, str) else x)
<ipython-input-2-50d803100278>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: replace_placeholders(x, api_response) if isinstance(x, str) else x)
<ipython-input-2-50d803100278>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: replace_placeholders(x, api_response) if isinstance(x, str) else x)
<ipython-input-2-50d803100278>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x

In [6]:
saving_file(final_df_people)
saving_file(final_df_employment)
saving_file(final_df_housing)
saving_file(final_df_income)
saving_file(final_df_education)

Data written/appended to scraped_data.xlsx
Data written/appended to scraped_data.xlsx
Data written/appended to scraped_data.xlsx
Data written/appended to scraped_data.xlsx
Data written/appended to scraped_data.xlsx
